In [19]:
Country_Tags={'TH - Thailand':'thailand','NO - Norway':'norway','NZ - New Zealand':'new zealand', 'HK - Hong Kong':'hong kong','BR - Brazil':'brazil', 
              'CE - Sri Lanka':'sri lanka','SW - Sweden':'sweden','AS - Australia':'australia','CH - China (Mainland)':'china','VE - Venezuela':'venezuela',
              'UK - United Kingdom':'united kingdom','TW - Taiwan':"taiwan",'CA - Canada':'canada','DA - Denmark':'denmark','IN - India':'india','JA - Japan; Okinawa;':'japan',
              'KS - Korea (South)':'korea','MY - Malaysia':'malaysia','MX - Mexico':'mexico',
              'SF - South Africa':'south africa','SN - Singapore':'singapore'}

In [20]:
keys_to_delete=['Raw content', 'Metadata', 'Canonical ID', 'Original Classification', 
                'Current Classification', 'Handling Restrictions', 'Character Count', 'Executive Order', 
                'Locator', 'Concepts', 'Enclosure', 'Type', 'Office Origin', 'Office Action', 
                'Archive Status', 'From', 'Markings', 'To', 'text2vec', "TAGS"]

In [21]:
def del_add_country_list(filename):
    ##load data
    file=open(file_name)
    data = json.load(file)
    file.close()
    ##del cable that cannot match
    del_list=set()
    for key in data.keys():
        #print (key+":")
        found_country_tag=False
        if "TAGS" not in data[key].keys():
            return
        for tag in data[key]["TAGS"].split("|"):
            Tag=tag.strip()
            if Tag in Country_Tags:
                found_country_tag=True
        if (not found_country_tag):
            del_list.add(key)
    ##del keys
    for item in del_list:
        del data[item]
    ##set key to 0
    for key in data.keys():
        for Country in Country_Tags.keys():
            data[key][Country_Tags[Country]]=0
        try:
            for tag in data[key]["TAGS"].split("|"):
                Tag=tag.strip()
                if Tag in Country_Tags.keys():
                    data[key][Country_Tags[Tag]]=1
        except:
            pass
    ##del text2vec key
    #     for key in data.keys():
#         for key_delete in keys_to_delete:
#             try:
#                 del data[key][key_delete]
#             except:
#                 pass
    with open('./output/data_matching_{}'.format(file_name[13:]), 'w') as fp:
        json.dump(data, fp)

In [22]:
import glob 
import json
for file_name in glob.glob("*.json*"):
    print ("Currently working on:", file_name)
    del_add_country_list(file_name)

Currently working on: data_crawled_2010.json


In [29]:
!ls

Country_Matching_Sever_Version.ipynb data_crawled_2010.json
Matching Data                        output
Model_dataset                        split_data.py
Random_ Forest_Model.ipynb


In [33]:
import pandas as pd 
match = pd.read_json(open('./Matching Data/output_mergedasDF/merge_dataframe.json').read())

In [36]:
len(match)

40607

In [45]:
japan=pd.read_json("./Archive/final_taiwan")

In [48]:
japan["Content"]

0       SHUI-BIAN'S NEW YEAR'S ADDRESS \n \n \nSummary...
1        \n \nA)  "No Warning in Armitage Comments" \n...
10       \n1. (C) Summary.  Chinese President Hu Jinta...
100     (d). \n \n1. (C) SUMMARY. In a June 30 farewel...
1000         B. MIKULAK-NEELY EMAIL OF 12/20/2006 \n \...
1001     \n \n1.  Director Stephen M. Young requests a...
1002    PRESIDENT CHEN SHUI-BIAN'S "FOUR WANTS" REMARK...
1003         B. TAIPEI 2682 \n     C. TAIPEI 2475 \n  ...
1004    PRESIDENT CHEN SHUI-BIAN'S "FOUR WANTS" REMARK...
1005     \n \n1.  SUMMARY:  Taiwan's current account s...
1006     \nClassified By: AIT Director Stephen M. Youn...
1007     \n 1. SUMMARY.  On March 2 the Legislative Yu...
1008    Could Feed Concerns of Dependence \n \nRef: Ta...
1009    Robert Griffiths.  Reasons 1.4 (b/d). \n \n1. ...
101     1.4 (b) and (d). \n \n \nSummary \n------- \n ...
1010    AP AND CNN \n \n \n1.  Summary:  Taiwan's news...
1011     \n1. (S/NOFORN) Summary. During a March 13 me...
1012    Talks;

In [49]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

binary_vectorizer=CountVectorizer(binary = True, ngram_range = (1, 2))
binary_vectorizer.fit(japan["Content"])


CountVectorizer(analyzer='word', binary=True, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 2), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [50]:
X_train_binary = binary_vectorizer.transform(japan["Content"])

In [54]:
tfidf_vectorizer=TfidfVectorizer (binary =True, ngram_range = (1, 2))
X_train_tfidf=tfidf_vectorizer.fit(japan["Content"])
sparse_matrix=X_train_tfidf.transform(japan["Content"])

In [66]:
japan["Content"][4]

' \n1.  (C) Summary.  According to Taiwan\'s Mainland Affairs \nCouncil (MAC), Beijing this afternoon, January 13,  accepted \nTaiwan\'s proposed delegation to participate in negotiations \non direct cross-Strait charter flights around the Chinese New \nYear.  At this still early point in the negotiation process, \nboth sides have actually made small concessions -- Beijing by \napparently accepting the presence of Taiwan government \naviation officials in the Taiwan delegation, and Taiwan by \npulling back from its adamant insistence that Taiwanese \nofficials responsible for cross-Strait relations be included \nin the Taiwan delegation.  In the face of growing Taiwan \npublic hopes for a charter flight deal in time for the \nChinese New Year, and of opposition Kuomintang (KMT) \ndemonstration that its delegation to Beijing successfully \nworked out arrangements for charter flights, the DPP \ngovernment had been forced to accept Beijing and the KMT\'s \nagenda, and received a small con

In [69]:
for name in X_train_tfidf.get_feature_names():
    if "\n" in name:
        print ("**")

In [58]:
tfidf = TfidfVectorizer(vocabulary = X_train_tfidf, stop_words = 'english', ngram_range=(1,2))

In [59]:
tfidf.get_feature_names()

NotFittedError: TfidfVectorizer - Vocabulary wasn't fitted.